In [5]:
# importações
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from CONSTANTES import *
from simulacao import *

In [6]:
opcoes = list(SIMULE.keys())
for modelo in opcoes:
    main(modelo)


# Iniciando simulações do Modelo Erdos-renyi #

Rede Social -- twitter-pequeno
N: 23370, <k>: 2.8097
Tempo com a rede: 0.06 segs.

Començando as simulações para a rede twitter-pequeno 

Twitter-pequeno 	Simulação #1 	OK
Twitter-pequeno 	Medidas #1 	OK
Twitter-pequeno 	Simulação #4 	OK
Twitter-pequeno 	Medidas #4 	OK
Twitter-pequeno 	Simulação #3 	OK
Twitter-pequeno 	Medidas #3 	OK
Twitter-pequeno 	Simulação #5 	OK
Twitter-pequeno 	Medidas #5 	OK
Twitter-pequeno 	Simulação #2 	OK
Twitter-pequeno 	Medidas #2 	OK
Twitter-pequeno 	Simulação #6 	OK
Twitter-pequeno 	Medidas #6 	OK
Twitter-pequeno 	Simulação #8 	OK
Twitter-pequeno 	Medidas #8 	OK
Twitter-pequeno 	Simulação #7 	OK
Twitter-pequeno 	Medidas #7 	OK
Twitter-pequeno 	Simulação #9 	OK
Twitter-pequeno 	Medidas #9 	OK
Twitter-pequeno 	Simulação #10 	OK
Twitter-pequeno 	Medidas #10 	OK

Fim das simulações para a rede twitter-pequeno
Tempo de simulação: 0 mins e 50.75 segs.

Fim de todas as simulações. Tempo total: 0 mins e 50.75 segs.

In [2]:
# teste:
for rede in redes_sociais.keys():

    ######################################
    # lidando com a rede social
    ######################################
    print('Rede Social --', rede)

    G = nx.read_edgelist("redes/" + redes_sociais[rede], comments='%', nodetype=int)
    grau_medio = np.array(G.degree)[:, 1].mean()
    N = len(G)
    print('<k>:', grau_medio, '\tN:', N)

Rede Social -- twitter-pequeno
<k>: 2.809670517757809 	N: 23370


In [7]:
# teste de simulação:
aux = nx.erdos_renyi_graph(N, grau_medio/(N-1), seed=42)

# plt.figure(figsize=(12,8))
# pos = nx.spring_layout(aux, iterations=30, k=0.5)
# nx.draw(aux, node_size=20, with_labels=False)

MEDIDAS['grau_medio'](aux)

2.8125802310654686